In [4]:
from transformers import AutoConfig

In [60]:
from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained("openai-community/gpt2")
config.tie_word_embeddings = False  # Disable weight tying


In [61]:
config.n_layer

12

In [63]:
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel(config)

In [77]:
model.lm_head

Linear(in_features=768, out_features=50257, bias=False)

In [11]:
# model.transformer.h[0]
model.lm_head

Linear(in_features=768, out_features=50257, bias=False)

In [25]:
from safetensors.torch import load_file

state_dict = load_file("../../../data/model.safetensors")

In [26]:
from safetensors import safe_open

stage_sd = {}
owm_key = [model.transformer.h[0], model.transformer.h[1], model.transformer.ln_f]
with safe_open("../../../data/model.safetensors", framework="pt") as f:
    for k in f.keys():
        print(k)
        # if owns_key(k):
        #     stage_sd[k] = f.get_tensor(k)

h.0.attn.bias
h.0.attn.c_attn.bias
h.0.attn.c_attn.weight
h.0.attn.c_proj.bias
h.0.attn.c_proj.weight
h.0.ln_1.bias
h.0.ln_1.weight
h.0.ln_2.bias
h.0.ln_2.weight
h.0.mlp.c_fc.bias
h.0.mlp.c_fc.weight
h.0.mlp.c_proj.bias
h.0.mlp.c_proj.weight
h.1.attn.bias
h.1.attn.c_attn.bias
h.1.attn.c_attn.weight
h.1.attn.c_proj.bias
h.1.attn.c_proj.weight
h.1.ln_1.bias
h.1.ln_1.weight
h.1.ln_2.bias
h.1.ln_2.weight
h.1.mlp.c_fc.bias
h.1.mlp.c_fc.weight
h.1.mlp.c_proj.bias
h.1.mlp.c_proj.weight
h.10.attn.bias
h.10.attn.c_attn.bias
h.10.attn.c_attn.weight
h.10.attn.c_proj.bias
h.10.attn.c_proj.weight
h.10.ln_1.bias
h.10.ln_1.weight
h.10.ln_2.bias
h.10.ln_2.weight
h.10.mlp.c_fc.bias
h.10.mlp.c_fc.weight
h.10.mlp.c_proj.bias
h.10.mlp.c_proj.weight
h.11.attn.bias
h.11.attn.c_attn.bias
h.11.attn.c_attn.weight
h.11.attn.c_proj.bias
h.11.attn.c_proj.weight
h.11.ln_1.bias
h.11.ln_1.weight
h.11.ln_2.bias
h.11.ln_2.weight
h.11.mlp.c_fc.bias
h.11.mlp.c_fc.weight
h.11.mlp.c_proj.bias
h.11.mlp.c_proj.weight
h.2.at

In [27]:
layers = []
for layer in model.transformer.h:
            layers.append(layer)  

In [28]:
layers

[GPT2Block(
   (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
   (attn): GPT2Attention(
     (c_attn): Conv1D(nf=2304, nx=768)
     (c_proj): Conv1D(nf=768, nx=768)
     (attn_dropout): Dropout(p=0.1, inplace=False)
     (resid_dropout): Dropout(p=0.1, inplace=False)
   )
   (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
   (mlp): GPT2MLP(
     (c_fc): Conv1D(nf=3072, nx=768)
     (c_proj): Conv1D(nf=768, nx=3072)
     (act): NewGELUActivation()
     (dropout): Dropout(p=0.1, inplace=False)
   )
 ),
 GPT2Block(
   (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
   (attn): GPT2Attention(
     (c_attn): Conv1D(nf=2304, nx=768)
     (c_proj): Conv1D(nf=768, nx=768)
     (attn_dropout): Dropout(p=0.1, inplace=False)
     (resid_dropout): Dropout(p=0.1, inplace=False)
   )
   (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
   (mlp): GPT2MLP(
     (c_fc): Conv1D(nf=3072, nx=768)
     (c_proj): Conv1D(nf=768, nx=3072)
     (act): NewGE

In [29]:
layers_per_node = 6
local_rank = 0
out_layers = layers[layers_per_node * local_rank : ((layers_per_node * local_rank) + layers_per_node) - 1]

In [97]:
from transformers import AutoConfig
from typing import List
import torch
from safetensors import safe_open

def get_model_per_node(config: AutoConfig, model, num_nodes: int, local_rank: int, model_name: str, weights_path: str) -> List:
    
    out_layers = {}
    results = []
    if model_name == 'causal_gpt2':
        
        total_layers = config.n_layer
        
        layers_per_node = total_layers // num_nodes
        
        layers = []
        
        for layer in model.transformer.h:
            layers.append(layer)     
            
        if local_rank == 0:
            out_layers['model.transformer.wte'] = model.transformer.wte
            out_layers['model.transformer.wpe'] = model.transformer.wpe
            
        
        elif local_rank == num_nodes - 1:
            selected_layers = layers[layers_per_node * local_rank : ((layers_per_node * local_rank) + layers_per_node) - 1]
            
            for layer in selected_layers:
                out_layers[f'model.transformer.h.{layers.index(layer)}'] = layer
                
            out_layers['model.transformer.ln_f'] = model.transformer.ln_f   
            
            out_layers['model.lm_head'] = model.lm_head
        
        
        else:
            selected_layers = layers[layers_per_node * local_rank : ((layers_per_node * local_rank) + layers_per_node) - 1]
            
            for layer in selected_layers:
                out_layers[f'model.transformer.h.{layers.index(layer)}'] = layer
        
        for layer_name, layer in out_layers.items():
            for i in layer.named_parameters():
                if layer_name == 'model.lm_head':
                    results.append(layer_name.split('model.')[1] + '.' + i[0])
                else:
                    results.append(layer_name.split('model.transformer.')[1] + '.' + i[0])
        
        
                # When building the state dict, remap the keys to match ModuleList indexing
       
        stage_sd = {}
        layer_mapping = {}  # Map original layer index to new ModuleList index

        print(out_layers)
        # Build mapping for layers
        modulelist_idx = 0
        for layer_name, layer in out_layers.items():
            if 'h.' in layer_name:
                # Extract original layer number (e.g., 'model.transformer.h.9' -> 9)
                original_idx = int(layer_name.split('h.')[1])
                layer_mapping[original_idx] = modulelist_idx
                modulelist_idx += 1
            elif 'ln_f' in layer_name:
                # ln_f is the last layer in ModuleList
                layer_mapping['ln_f'] = modulelist_idx
                modulelist_idx += 1
            elif 'wte' in layer_name:
                layer_mapping['wte'] = modulelist_idx
                modulelist_idx += 1
            elif 'wpe' in layer_name:
                layer_mapping['wpe'] = modulelist_idx
                modulelist_idx += 1
            elif 'lm_head' in layer_name:
                layer_mapping['lm_head'] = modulelist_idx
                modulelist_idx += 1
        
        # Load weights with remapped keys
        with safe_open(weights_path, framework="pt") as f:
            for k in results:
                for key in f.keys():
                    if k == key:
                        # Remap the key to match ModuleList indexing
                        if 'h.' in k:
                            original_idx = int(k.split('h.')[1].split('.')[0])
                            new_idx = layer_mapping[original_idx]
                            # Replace h.9 with 0, h.10 with 1, etc.
                            new_key = k.replace(f'h.{original_idx}', str(new_idx))
                        elif 'ln_f' in k:
                            new_key = k.replace('ln_f', str(layer_mapping['ln_f']))
                        elif 'wpe' in k:
                            new_key = k.replace('wpe', str(layer_mapping['wpe']))
                        elif 'wte' in k:
                            new_key = k.replace('wte', str(layer_mapping['wte']))
                        elif 'lm_head' in k:
                            new_key = k.replace('lm_head', str(layer_mapping['lm_head']))
                        else:
                            new_key = k
                        
                        stage_sd[new_key] = f.get_tensor(k)

        # Now load into ModuleList
        final_layers = list(out_layers.values())
        final_model = torch.nn.ModuleList(final_layers)
        loaded_model = final_model.load_state_dict(stage_sd, strict=False)

        # Return the model, not the load_state_dict result
        return final_model

In [98]:
modelA  = get_model_per_node(config, model, num_nodes=4, local_rank=3, model_name='causal_gpt2', weights_path="../../../data/model.safetensors")

{'model.transformer.h.9': GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D(nf=2304, nx=768)
    (c_proj): Conv1D(nf=768, nx=768)
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D(nf=3072, nx=768)
    (c_proj): Conv1D(nf=768, nx=3072)
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
), 'model.transformer.h.10': GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D(nf=2304, nx=768)
    (c_proj): Conv1D(nf=768, nx=768)
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D(nf=3072, nx=768)
    (c_proj): Conv1D(nf=768, nx

In [80]:
out_layers


NameError: name 'out_layers' is not defined

In [33]:

results = []

In [41]:
for layer_name, layer in out_layers.items():
    for i in layer.named_parameters():
        results.append(layer_name.split('model.transformer.')[1] + '.' + i[0])

In [42]:
results

['h.9.ln_1.weight',
 'h.9.ln_1.bias',
 'h.9.attn.c_attn.weight',
 'h.9.attn.c_attn.bias',
 'h.9.attn.c_proj.weight',
 'h.9.attn.c_proj.bias',
 'h.9.ln_2.weight',
 'h.9.ln_2.bias',
 'h.9.mlp.c_fc.weight',
 'h.9.mlp.c_fc.bias',
 'h.9.mlp.c_proj.weight',
 'h.9.mlp.c_proj.bias',
 'h.10.ln_1.weight',
 'h.10.ln_1.bias',
 'h.10.attn.c_attn.weight',
 'h.10.attn.c_attn.bias',
 'h.10.attn.c_proj.weight',
 'h.10.attn.c_proj.bias',
 'h.10.ln_2.weight',
 'h.10.ln_2.bias',
 'h.10.mlp.c_fc.weight',
 'h.10.mlp.c_fc.bias',
 'h.10.mlp.c_proj.weight',
 'h.10.mlp.c_proj.bias',
 'ln_f.weight',
 'ln_f.bias',
 'h.9.ln_1.weight',
 'h.9.ln_1.bias',
 'h.9.attn.c_attn.weight',
 'h.9.attn.c_attn.bias',
 'h.9.attn.c_proj.weight',
 'h.9.attn.c_proj.bias',
 'h.9.ln_2.weight',
 'h.9.ln_2.bias',
 'h.9.mlp.c_fc.weight',
 'h.9.mlp.c_fc.bias',
 'h.9.mlp.c_proj.weight',
 'h.9.mlp.c_proj.bias',
 'h.10.ln_1.weight',
 'h.10.ln_1.bias',
 'h.10.attn.c_attn.weight',
 'h.10.attn.c_attn.bias',
 'h.10.attn.c_proj.weight',
 'h.10.a

In [20]:
from safetensors import safe_open

stage_sd = {}
with safe_open("../../../data/model.safetensors", framework="pt") as f:
    # for k in results:
    for key in f.keys():
        print(key)
        # if k == key:
            # if k in results:
            # key = 'transformer.' + k
            # stage_sd[key] = f.get_tensor(k)
    #       stage_sd[k] = f.get_tensor(k)

h.0.attn.bias
h.0.attn.c_attn.bias
h.0.attn.c_attn.weight
h.0.attn.c_proj.bias
h.0.attn.c_proj.weight
h.0.ln_1.bias
h.0.ln_1.weight
h.0.ln_2.bias
h.0.ln_2.weight
h.0.mlp.c_fc.bias
h.0.mlp.c_fc.weight
h.0.mlp.c_proj.bias
h.0.mlp.c_proj.weight
h.1.attn.bias
h.1.attn.c_attn.bias
h.1.attn.c_attn.weight
h.1.attn.c_proj.bias
h.1.attn.c_proj.weight
h.1.ln_1.bias
h.1.ln_1.weight
h.1.ln_2.bias
h.1.ln_2.weight
h.1.mlp.c_fc.bias
h.1.mlp.c_fc.weight
h.1.mlp.c_proj.bias
h.1.mlp.c_proj.weight
h.10.attn.bias
h.10.attn.c_attn.bias
h.10.attn.c_attn.weight
h.10.attn.c_proj.bias
h.10.attn.c_proj.weight
h.10.ln_1.bias
h.10.ln_1.weight
h.10.ln_2.bias
h.10.ln_2.weight
h.10.mlp.c_fc.bias
h.10.mlp.c_fc.weight
h.10.mlp.c_proj.bias
h.10.mlp.c_proj.weight
h.11.attn.bias
h.11.attn.c_attn.bias
h.11.attn.c_attn.weight
h.11.attn.c_proj.bias
h.11.attn.c_proj.weight
h.11.ln_1.bias
h.11.ln_1.weight
h.11.ln_2.bias
h.11.ln_2.weight
h.11.mlp.c_fc.bias
h.11.mlp.c_fc.weight
h.11.mlp.c_proj.bias
h.11.mlp.c_proj.weight
h.2.at

In [45]:
stage_sd

{'transformer.h.9.ln_1.weight': tensor([0.3815, 0.3605, 0.3697, 0.3585, 0.3653, 0.3277, 0.4366, 0.3190, 0.3760,
         0.3174, 0.3975, 0.3641, 0.3470, 0.3779, 0.3925, 0.3663, 0.3564, 0.3584,
         0.3682, 0.3501, 0.3349, 0.3449, 0.3698, 0.3721, 0.3585, 0.3598, 0.3786,
         0.3364, 0.3586, 0.3547, 0.3510, 0.3606, 0.3541, 0.3366, 0.3623, 0.3904,
         0.3035, 0.3562, 0.3467, 0.3565, 0.3333, 0.3324, 0.3350, 0.3506, 0.3571,
         0.3667, 0.3467, 0.3370, 0.3564, 0.3545, 0.3442, 0.3410, 0.3603, 0.3587,
         0.3565, 0.3505, 0.3562, 0.3838, 0.3350, 0.3870, 0.3623, 0.3577, 0.3859,
         0.3485, 0.9022, 0.3349, 0.3387, 0.3671, 0.3486, 0.3438, 0.3759, 0.2744,
         0.3469, 0.3436, 0.3389, 0.3589, 0.3936, 0.3936, 0.3719, 0.3413, 0.3468,
         0.3408, 0.3602, 0.3486, 0.3457, 0.3838, 0.3643, 0.7337, 0.3350, 0.3663,
         0.3487, 0.3469, 0.3343, 0.3448, 0.3369, 0.3408, 0.3642, 0.3808, 0.3825,
         0.3819, 0.3486, 0.3291, 0.3740, 0.3694, 0.3543, 0.3478, 0.3838, 0.387

In [46]:
out_layers

{'model.transformer.h.9': GPT2Block(
   (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
   (attn): GPT2Attention(
     (c_attn): Conv1D(nf=2304, nx=768)
     (c_proj): Conv1D(nf=768, nx=768)
     (attn_dropout): Dropout(p=0.1, inplace=False)
     (resid_dropout): Dropout(p=0.1, inplace=False)
   )
   (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
   (mlp): GPT2MLP(
     (c_fc): Conv1D(nf=3072, nx=768)
     (c_proj): Conv1D(nf=768, nx=3072)
     (act): NewGELUActivation()
     (dropout): Dropout(p=0.1, inplace=False)
   )
 ),
 'model.transformer.h.10': GPT2Block(
   (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
   (attn): GPT2Attention(
     (c_attn): Conv1D(nf=2304, nx=768)
     (c_proj): Conv1D(nf=768, nx=768)
     (attn_dropout): Dropout(p=0.1, inplace=False)
     (resid_dropout): Dropout(p=0.1, inplace=False)
   )
   (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
   (mlp): GPT2MLP(
     (c_fc): Conv1D(nf=3072, nx=768)
     

In [47]:
loaded_model_layers = []
final_layers = []
for i in out_layers:
   final_layers.append(out_layers[i])
final_model = torch.nn.ModuleList(final_layers)

loaded_model = final_model.load_state_dict(stage_sd, strict=False)

In [48]:
final_model

ModuleList(
  (0-1): 2 x GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GPT2Attention(
      (c_attn): Conv1D(nf=2304, nx=768)
      (c_proj): Conv1D(nf=768, nx=768)
      (attn_dropout): Dropout(p=0.1, inplace=False)
      (resid_dropout): Dropout(p=0.1, inplace=False)
    )
    (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (mlp): GPT2MLP(
      (c_fc): Conv1D(nf=3072, nx=768)
      (c_proj): Conv1D(nf=768, nx=3072)
      (act): NewGELUActivation()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)

In [49]:
loaded_model

_IncompatibleKeys(missing_keys=['0.ln_1.weight', '0.ln_1.bias', '0.attn.c_attn.weight', '0.attn.c_attn.bias', '0.attn.c_proj.weight', '0.attn.c_proj.bias', '0.ln_2.weight', '0.ln_2.bias', '0.mlp.c_fc.weight', '0.mlp.c_fc.bias', '0.mlp.c_proj.weight', '0.mlp.c_proj.bias', '1.ln_1.weight', '1.ln_1.bias', '1.attn.c_attn.weight', '1.attn.c_attn.bias', '1.attn.c_proj.weight', '1.attn.c_proj.bias', '1.ln_2.weight', '1.ln_2.bias', '1.mlp.c_fc.weight', '1.mlp.c_fc.bias', '1.mlp.c_proj.weight', '1.mlp.c_proj.bias', '2.weight', '2.bias'], unexpected_keys=['transformer.h.9.ln_1.weight', 'transformer.h.9.ln_1.bias', 'transformer.h.9.attn.c_attn.weight', 'transformer.h.9.attn.c_attn.bias', 'transformer.h.9.attn.c_proj.weight', 'transformer.h.9.attn.c_proj.bias', 'transformer.h.9.ln_2.weight', 'transformer.h.9.ln_2.bias', 'transformer.h.9.mlp.c_fc.weight', 'transformer.h.9.mlp.c_fc.bias', 'transformer.h.9.mlp.c_proj.weight', 'transformer.h.9.mlp.c_proj.bias', 'transformer.h.10.ln_1.weight', 'transfo

In [17]:
# When building the state dict, remap the keys to match ModuleList indexing
from safetensors import safe_open

# stage_sd = {}
layer_mapping = {}  # Map original layer index to new ModuleList index
out_layers = 
# Build mapping for layers
modulelist_idx = 0
for layer_name, layer in out_layers.items():
    if 'h.' in layer_name:
        # Extract original layer number (e.g., 'model.transformer.h.9' -> 9)
        original_idx = int(layer_name.split('h.')[1])
        layer_mapping[original_idx] = modulelist_idx
        modulelist_idx += 1
    elif 'ln_f' in layer_name:
        # ln_f is the last layer in ModuleList
        layer_mapping['ln_f'] = modulelist_idx
    elif 'wte' in layer_name:
        layer_mapping['wte'] = modulelist_idx
    elif 'wpe' in layer_name:
        layer_mapping['wpe'] = modulelist_idx
        
# Load weights with remapped keys
with safe_open("../../../data/model.safetensors", framework="pt") as f:
    for k in results:
        for key in f.keys():
            if k == key:
                # Remap the key to match ModuleList indexing
                if 'h.' in k:
                    original_idx = int(k.split('h.')[1].split('.')[0])
                    new_idx = layer_mapping[original_idx]
                    # Replace h.9 with 0, h.10 with 1, etc.
                    new_key = k.replace(f'h.{original_idx}', str(new_idx))
                elif 'ln_f' in k:
                    new_key = k.replace('ln_f', str(layer_mapping['ln_f']))
                else:
                    new_key = k
                
                stage_sd[new_key] = f.get_tensor(k)

# Now load into ModuleList
final_layers = list(out_layers.values())
final_model = torch.nn.ModuleList(final_layers)
loaded_model = final_model.load_state_dict(stage_sd, strict=False)

NameError: name 'out_layers' is not defined

In [53]:
loaded_model

<All keys matched successfully>

In [99]:
modelA

ModuleList(
  (0-1): 2 x GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GPT2Attention(
      (c_attn): Conv1D(nf=2304, nx=768)
      (c_proj): Conv1D(nf=768, nx=768)
      (attn_dropout): Dropout(p=0.1, inplace=False)
      (resid_dropout): Dropout(p=0.1, inplace=False)
    )
    (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (mlp): GPT2MLP(
      (c_fc): Conv1D(nf=3072, nx=768)
      (c_proj): Conv1D(nf=768, nx=3072)
      (act): NewGELUActivation()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (3): Linear(in_features=768, out_features=50257, bias=False)
)

In [100]:
lm_head = torch.nn.Linear(768, 50257, bias=False)

for module in hollo_modelA:
    if isinstance(module, torch.nn.Embedding):
        lm_head.weight = module.weight
        break
    
_modelA = torch.nn.ModuleList(list(hollo_modelA) + [lm_head])
_modelA.load_state_dict(loaded_modelA, strict=False)

_IncompatibleKeys(missing_keys=['3.weight'], unexpected_keys=[])

In [101]:
modelB = get_model_per_node(config, model, num_nodes=4, local_rank=2, model_name='causal_gpt2', weights_path="../../../data/model.safetensors")

{'model.transformer.h.6': GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D(nf=2304, nx=768)
    (c_proj): Conv1D(nf=768, nx=768)
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D(nf=3072, nx=768)
    (c_proj): Conv1D(nf=768, nx=3072)
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
), 'model.transformer.h.7': GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D(nf=2304, nx=768)
    (c_proj): Conv1D(nf=768, nx=768)
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D(nf=3072, nx=768)
    (c_proj): Conv1D(nf=768, nx=

In [102]:
modelB

ModuleList(
  (0-1): 2 x GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GPT2Attention(
      (c_attn): Conv1D(nf=2304, nx=768)
      (c_proj): Conv1D(nf=768, nx=768)
      (attn_dropout): Dropout(p=0.1, inplace=False)
      (resid_dropout): Dropout(p=0.1, inplace=False)
    )
    (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (mlp): GPT2MLP(
      (c_fc): Conv1D(nf=3072, nx=768)
      (c_proj): Conv1D(nf=768, nx=3072)
      (act): NewGELUActivation()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)

In [103]:
modelC = get_model_per_node(config, model, num_nodes=4, local_rank=1, model_name='causal_gpt2', weights_path="../../../data/model.safetensors")

{'model.transformer.h.3': GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D(nf=2304, nx=768)
    (c_proj): Conv1D(nf=768, nx=768)
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D(nf=3072, nx=768)
    (c_proj): Conv1D(nf=768, nx=3072)
    (act): NewGELUActivation()
    (dropout): Dropout(p=0.1, inplace=False)
  )
), 'model.transformer.h.4': GPT2Block(
  (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (attn): GPT2Attention(
    (c_attn): Conv1D(nf=2304, nx=768)
    (c_proj): Conv1D(nf=768, nx=768)
    (attn_dropout): Dropout(p=0.1, inplace=False)
    (resid_dropout): Dropout(p=0.1, inplace=False)
  )
  (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  (mlp): GPT2MLP(
    (c_fc): Conv1D(nf=3072, nx=768)
    (c_proj): Conv1D(nf=768, nx=

In [104]:
modelC

ModuleList(
  (0-1): 2 x GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GPT2Attention(
      (c_attn): Conv1D(nf=2304, nx=768)
      (c_proj): Conv1D(nf=768, nx=768)
      (attn_dropout): Dropout(p=0.1, inplace=False)
      (resid_dropout): Dropout(p=0.1, inplace=False)
    )
    (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (mlp): GPT2MLP(
      (c_fc): Conv1D(nf=3072, nx=768)
      (c_proj): Conv1D(nf=768, nx=3072)
      (act): NewGELUActivation()
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
)

In [105]:
modelD = get_model_per_node(config, model, num_nodes=4, local_rank=0, model_name='causal_gpt2', weights_path="../../../data/model.safetensors")

{'model.transformer.wte': Embedding(50257, 768), 'model.transformer.wpe': Embedding(1024, 768)}


In [106]:
modelD

ModuleList(
  (0): Embedding(50257, 768)
  (1): Embedding(1024, 768)
)

In [107]:
from transformers import AutoTokenizer
text = 'Hi my name is '
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
inputs = tokenizer(text, return_tensors="pt")
inputs['input_ids']

tensor([[17250,   616,  1438,   318,   220]])

In [119]:
x = modelD[0](inputs['input_ids'])
pos_embeds = torch.arange(x.shape[1], dtype=torch.long)
out = modelD[1](pos_embeds)
x = x + out

for layer in modelC:
    x = layer(x)

TypeError: layer_norm(): argument 'input' (position 1) must be Tensor, not tuple

In [ ]:
# Verify weights are loaded
for name, param in modelD.named_parameters():
    print(f"{name}: shape={param.shape}, mean={param.mean():.4f}, std={param.std():.4f}")
    break  # Just show first param to verify it's not random initialization